In [1]:
# Import necessary modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from selenium import webdriver
from datetime import datetime
import time
import os

from multiprocessing import current_process
from datetime import date, timedelta

## Test Area

In [2]:
td = datetime.today().strftime('%m_%d_%Y'); path = "./" + td + "/"
os.system( "mkdir {}".format(td) )
yest = datetime.today() - timedelta(1)
yest = yest.strftime('%m_%d_%Y')

In [3]:
nba_stats_info = {"https://www.sportsbookreview.com/betting-odds/nba-basketball/": \
                  [["/html/body/div/div/div/div/section/div/div[3]/div[1]/div[1]/ol/li[1]", \
                    "/html/body/div/div/div/div/section/div/div[3]/div[1]/div[2]/ol/li[2]"],\
                  "bettingOddsGridContainer",\
                  [8,10] + [i for i in range(12,20)] + [23],\
                  ['%FGA','%3PA','%FTA','%OREB','%DREB','%REB','%AST','%TOV','%STL','%BLK','%PTS'],\
                  18,\
                  3,\
                   "/html/body/div/div/div/div/section/div/div[3]/div[2]/div[3]/div[3]/div[1]/div/div"]}

In [5]:
## Unpack data
url = "https://www.sportsbookreview.com/betting-odds/nba-basketball/"
xpaths = nba_stats_info[url][0]
object_id = nba_stats_info[url][1]
stats_args = nba_stats_info[url][2]
cols = nba_stats_info[url][3]
dp = nba_stats_info[url][4]
fn = nba_stats_info[url][5]
click_arrow = nba_stats_info[url][6]

In [24]:
## Set up chrome driver, open browser
path_to_chromedriver = "/home/richie10181/anaconda3/RG_Ventures/SportsBetting/HistoricalData/Basketball/Odds/web_scraping/chromedriver"
browser = webdriver.Chrome(executable_path=path_to_chromedriver)
browser.get(url);
time.sleep(2)  ## make sure the browswer loads before executing xpaths

#cal = '/html/body/div/div/div/div/section/div/div[2]/div/div/span'
#day = '/html/body/div[5]/div/div/section/div/div/div[2]/div[2]/table/tbody/tr[5]/td[6]/div'
#browser.find_element_by_xpath(cal).click();
#time.sleep(2)
#browser.find_element_by_xpath(day).click();

# tr chunk represents rows (i.e. weeks of the month)
# td chunk represents days of week



In [29]:
## Use the xpaths to click the necessary browswer buttons
for xpath in xpaths:
    browser.find_element_by_xpath(xpath).click();
    time.sleep(2)  # Sleep in between

In [30]:
## Populate containers
table = browser.find_element_by_id(object_id)

In [31]:
# lists
teamNames = ['Phoenix','Orlando','Washington','Detroit','Indiana','Atlanta',
            'Toronto','Miami','Charlotte','Brooklyn','Cleveland','Memphis','Minnesota',
            'Chicago','New Orleans','Dallas','Denver','San Antonio','Sacramento','L.A. Clippers',
            'L.A. Lakers','Philadelphia','Boston', 'New York','Houston','Oklahoma City',
            'Golden State','Utah','Portland','Milwaukee']

# Containers
GameID = []
Date = []
Teams = []
Points = []
Wagers = []
Opener = []
Pinnacle = []
Fivedimes = []
Bookmaker = []
BetOnline = []

#Justbet = []
#Sportsbetting = []

#colnames = [GameID, Date, Teams, Points, Wagers, Pinnacle, Fivedimes, Bookmaker, BetOnline,
#           Bovada, Heritage, Intertops, Youwager, Justbet, Sportsbetting]

colnames = [GameID, Date, Teams, Points, Wagers, Opener, Pinnacle, Fivedimes, Bookmaker, BetOnline]

            
data = table.text.split("\n")[dp:]

In [32]:
data

['Mon Mar 10, 2014',
 'WAGERS',
 'OPENER',
 '07:00 PM',
 'H2H',
 '|',
 '501',
 'Denver',
 '502',
 'Charlotte',
 '22',
 '25',
 '23',
 '26',
 '23',
 '28',
 '30',
 '26',
 '98',
 '105',
 '-',
 '-',
 '+195',
 '-215',
 '+193',
 '-215',
 '+190',
 '-210',
 '+192',
 '-230',
 '+190',
 '-220',
 '07:30 PM',
 'H2H',
 '|',
 '503',
 'Washington',
 '504',
 'Miami',
 '23',
 '31',
 '22',
 '22',
 '28',
 '20',
 '17',
 '26',
 '90',
 '99',
 '-',
 '-',
 '+300',
 '-330',
 '+346',
 '-400',
 '+330',
 '-370',
 '+320',
 '-400',
 '+330',
 '-400',
 '07:30 PM',
 'H2H',
 '|',
 '505',
 'Philadelphia',
 '506',
 'New York',
 '31',
 '26',
 '30',
 '35',
 '24',
 '30',
 '25',
 '32',
 '110',
 '123',
 '-',
 '-',
 '+205',
 '-2250',
 '+1350',
 '-2350',
 '+1400',
 '-1800',
 '+1300',
 '-2500',
 '+1500',
 '-3000',
 '07:30 PM',
 'H2H',
 '|',
 '507',
 'Toronto',
 '508',
 'Brooklyn',
 '30',
 '19',
 '21',
 '31',
 '22',
 '28',
 '24',
 '23',
 '97',
 '101',
 '-',
 '-',
 '+115',
 '-125',
 '+140',
 '-155',
 '+140',
 '-150',
 '+135',
 '-160

In [14]:
def populate(data, colnames, teamNames):
    game_id = 0
    team_count = 0
    odds_count = 0
    bookie_count = 0
    dt = data[0]
    
    for idx, elm in enumerate(data):
        
        # set data to first item in data list
        #if idx == 0: dt = elm

        
        # if element is a valid team name
        if elm in teamNames:
            colnames[1].append(dt)
            colnames[2].append(elm)
            colnames[0].append(game_id)
            team_count += 1
            
            # increment game_id once two teams are selected
            if team_count % 2 == 0:
                game_id += 1
                odds_count = 0
                bookie_count = 0
        
        # to account for overtime games, increment to the "wager" field
        # and then count backward to get total points
        if elm[-1] == "%" and data[idx-1][-1] == "%":
            colnames[3].append(data[idx-3])
            colnames[3].append(data[idx-2])
            
        if elm[-1] == "%":
            colnames[4].append(elm)

        #if idx >= 2:
        #    if data[idx-2][-1] == "%" and data[idx-1][-1] == "%":
                
        if elm[0] == '+' or elm[0] == '-':
            colnames[5+bookie_count].append(elm)
            odds_count += 1
            
            if odds_count % 2 == 0 and bookie_count < 4:
                bookie_count +=1
                
    return colnames

In [15]:
pop = populate(data,colnames,teamNames)

NameError: name 'data' is not defined

In [16]:
dfout = pd.DataFrame(pop,
                    index= ['GameID', 'Date', 'Teams', 'Points', 'Wagers',
                              'Opener', 'Pinnacle', 'Fivedimes', 'Bookmaker', 'BetOnline']).T

NameError: name 'pop' is not defined

In [12]:
dfout

GameID              Date          Teams Points Wagers Opener Pinnacle  \
0       0  Wed Dec 26, 2018        Phoenix    122    53%   +155     +185   
1       0  Wed Dec 26, 2018        Orlando    120    47%   -175     -206   
2       1  Wed Dec 26, 2018     Washington     95    52%   +175     +163   
3       1  Wed Dec 26, 2018        Detroit    106    48%   -210     -181   
4       2  Wed Dec 26, 2018        Indiana    129    54%   -345     -263   
5       2  Wed Dec 26, 2018        Atlanta    121    46%   +285     +233   
6       3  Wed Dec 26, 2018        Toronto    106    49%   -180     -122   
7       3  Wed Dec 26, 2018          Miami    104    51%   +158     +111   
8       4  Wed Dec 26, 2018      Charlotte    132    49%   +110     +101   
9       4  Wed Dec 26, 2018       Brooklyn    134    51%   -130     -111   
10      5  Wed Dec 26, 2018      Cleveland     87    48%   +425     +396   
11      5  Wed Dec 26, 2018        Memphis     95    52%   -550     -464   
12      6  Wed Dec 26, 2018      Minnesota    119    54%   -190     -171   
13      6  Wed Dec 26, 2018        Chicago     94    46%   +165     +154   
14      7  Wed Dec 26, 2018    New Orleans    119    48%   +115     +123   
15      7  Wed Dec 26, 2018         Dallas    122    52%   -135     -136   
16      8  Wed Dec 26, 2018         Denver    103    51%   +135     +154   
17      8  Wed Dec 26, 2018    San Antonio    111    49%   -155     -171   
18      9  Wed Dec 26, 2018     Sacramento    118    44%   +175     +177   
19      9  Wed Dec 26, 2018  L.A. Clippers    127    56%   -210     -197   

   Fivedimes Bookmaker BetOnline  
0       +190      +188      +180  
1       -210      -225      -210  
2       +165      +160      +163  
3       -175      -185      -183  
4       -265      -280      -265  
5       +245      +230      +225  
6       -125      -125      -130  
7       +115      +105      +110  
8       +102      -105      -102  
9       -112      -115      -118  
10      +420      +385      +420  
11      -475      -501      -515  
12      -170      -180      -175  
13      +160      +155      +155  
14      +120      +120      +115  
15      -130      -140      -135  
16      +160      +151      +157  
17      -170      -175      -177  
18      -125      +172      +175  
19      -115      -199      -205

## test concatenation of more bookies

## think of how to utiilize the "click_arrow" path
## maybe we run the process "as is" above, then click the arrow,
## and basically run the process again and append the columns

In [5]:
## Set up chrome driver, open browser
path_to_chromedriver = "./web_scraping/chromedriver"
browser = webdriver.Chrome(executable_path=path_to_chromedriver)
browser.get(url);
time.sleep(3)  ## make sure the browswer loads before executing xpaths

## Use the xpaths to click the necessary browswer buttons
for xpath in xpaths:
    browser.find_element_by_xpath(xpath).click();
    time.sleep(3)  # Sleep in between
    
browser.find_element_by_xpath(click_arrow).click();
time.sleep(3)  # Sleep in between

NameError: name 'webdriver' is not defined

In [22]:
##
## PUT THIS BEFORE CLOSE BROWSET
##
click_arrow2 = 'html/body/div/div/div/div/section/div/div[3]/div[2]/div[3]/div[2]/div/div/div[2]'

# click again
browser.find_element_by_xpath(click_arrow2).click();
table_after2 = browser.find_element_by_id(object_id)

In [23]:
data_after2 = table_after2.text.split("\n")

In [26]:
data_after2[:45]

['NBA Money Lines for Dec 26, 2018',
 'Full Game',
 '1st Half',
 '2nd Half',
 '1Q',
 '2Q',
 '3Q',
 '4Q',
 'Futures',
 'Point Spreads',
 'Money Lines',
 'Totals',
 'Spreads & Totals',
 'Box Scores',
 'Time',
 '|',
 'TV',
 'Rot',
 'Wed Dec 26, 2018',
 'WAGERS',
 'OPENER',
 '07:00 PM',
 'H2H',
 '|',
 '567',
 'Phoenix',
 '122',
 '568',
 'Orlando',
 '120',
 '53%',
 '47%',
 '+155',
 '-175',
 '+188',
 '-225',
 '+180',
 '-210',
 '+190',
 '-220',
 '+190',
 '-210',
 '07:00 PM',
 'H2H',
 '|']

In [28]:
# Containers
Opener3 = []
Justbet = []
Sportsbetting = []
Gtbets = []
Sportbet = []

colnames3 = [Opener3, Justbet, Sportsbetting, Gtbets, Sportbet]

In [29]:
# will need new populate function
def populate_after_click(data, colnames,teamNames):
    odds_count = 0
    bookie_count = 0
    team_count = 0
    
    for idx, elm in enumerate(data):
        
        # if element is a valid team name
        if elm in teamNames:
            team_count += 1
            
            # increment game_id once two teams are selected
            if team_count % 2 == 0:
                odds_count = 0
                bookie_count = 0
                
        if elm[0] == '+' or elm[0] == '-':
            colnames[bookie_count].append(elm)
            odds_count += 1
            
            if odds_count % 2 == 0 and bookie_count < 4:
                bookie_count +=1
                
    return colnames

In [31]:
pop_after2 = populate_after_click(data_after2,colnames3,teamNames)

In [32]:
dfout_after2 = pd.DataFrame(pop_after2,
                    index= ['Opener3','Justbet', 'Sportsbetting', 'Gtbets', 'Sportbet']).T

In [33]:
dfout_after2

Opener3 Justbet Sportsbetting Gtbets Sportbet
0     +155    +188          +180   +190     +190
1     -175    -225          -210   -220     -210
2     +175    +160          +163   +165     +165
3     -210    -185          -183   -185     -175
4     -345    -280          -265   -275     -265
5     +285    +230          +225   +235     +245
6     -180    -125          -130   -145     -125
7     +158    +105          +110   +125     +115
8     +110    -105          -102   -103     +102
9     -130    -115          -118   -117     -112
10    +425    +385          +420   +400     +420
11    -550    -501          -515   -500     -475
12    -190    -180          -175   -175     -170
13    +165    +155          +155   +155     +160
14    +115    +120          +115   +110     +120
15    -135    -140          -135   -130     -130
16    +135    +151          +157   +155     +160
17    -155    -175          -177   -175     -170
18    +175    +172          +175   +170     -125
19    -210    -199          -205   -200     -115

## Import File and Run

In [2]:
from scraping_nba_odds import nba_stats_info, scrape_nba_odds

In [3]:
url = list(nba_stats_info.keys())[0]

In [4]:
xstart = '/html/body/div[5]/div/div/section/div/div/div[2]/div[2]/table/tbody/tr['
xmid = ']/td['
xend = ']/div'
week = [1,2,3,4,5,6]
day = [1,2,3,4,5,6,7]
xpaths = [xstart + str(wk) + xmid + str(d) + xend for wk in week[::-1] for d in day[::-1]]

In [5]:
num_months = 4

In [7]:
dfout = scrape_nba_odds(url, xpaths, num_months)

--- Logging error ---
Traceback (most recent call last):
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 992, in emit
    msg = self.format(record)
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 838, in format
    return fmt.format(record)
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 575, in format
    record.message = record.getMessage()
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 338, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/home/richie10181/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/richie10181/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/richie10181/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()


--- Logging error ---
Traceback (most recent call last):
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 992, in emit
    msg = self.format(record)
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 838, in format
    return fmt.format(record)
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 575, in format
    record.message = record.getMessage()
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 338, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/home/richie10181/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/richie10181/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/richie10181/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()


--- Logging error ---
Traceback (most recent call last):
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 992, in emit
    msg = self.format(record)
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 838, in format
    return fmt.format(record)
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 575, in format
    record.message = record.getMessage()
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 338, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/home/richie10181/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/richie10181/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/richie10181/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()


--- Logging error ---
Traceback (most recent call last):
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 992, in emit
    msg = self.format(record)
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 838, in format
    return fmt.format(record)
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 575, in format
    record.message = record.getMessage()
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 338, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/home/richie10181/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/richie10181/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/richie10181/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()


--- Logging error ---
Traceback (most recent call last):
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 992, in emit
    msg = self.format(record)
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 838, in format
    return fmt.format(record)
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 575, in format
    record.message = record.getMessage()
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 338, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/home/richie10181/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/richie10181/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/richie10181/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()


...

--- Logging error ---
Traceback (most recent call last):
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 992, in emit
    msg = self.format(record)
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 838, in format
    return fmt.format(record)
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 575, in format
    record.message = record.getMessage()
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 338, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/home/richie10181/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/richie10181/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/richie10181/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()


--- Logging error ---
Traceback (most recent call last):
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 992, in emit
    msg = self.format(record)
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 838, in format
    return fmt.format(record)
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 575, in format
    record.message = record.getMessage()
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 338, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/home/richie10181/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/richie10181/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/richie10181/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()


--- Logging error ---
Traceback (most recent call last):
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 992, in emit
    msg = self.format(record)
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 838, in format
    return fmt.format(record)
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 575, in format
    record.message = record.getMessage()
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 338, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/home/richie10181/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/richie10181/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/richie10181/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()


--- Logging error ---
Traceback (most recent call last):
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 992, in emit
    msg = self.format(record)
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 838, in format
    return fmt.format(record)
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 575, in format
    record.message = record.getMessage()
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 338, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/home/richie10181/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/richie10181/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/richie10181/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()


--- Logging error ---
Traceback (most recent call last):
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 992, in emit
    msg = self.format(record)
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 838, in format
    return fmt.format(record)
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 575, in format
    record.message = record.getMessage()
  File "/home/richie10181/anaconda3/lib/python3.6/logging/__init__.py", line 338, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/home/richie10181/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/richie10181/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/richie10181/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()


## why is it not selecting date?

In [8]:
dfout = dfout.drop(['Opener2','Opener3', 'Opener4','Opener5','Opener6','Opener7'],axis=1)

In [9]:
dfout.head()

GameID              Date         Teams Points Wagers Opener Pinnacle  \
0      0  Sun Jan 13, 2019  Philadelphia    108    55%   -290     -415   
1      0  Sun Jan 13, 2019      New York    105    45%   +230     +358   
2      1  Sun Jan 13, 2019       Toronto    140    45%   -280     -202   
3      1  Sun Jan 13, 2019    Washington    138    55%   +240     +181   
4      2  Sun Jan 13, 2019     Milwaukee    133    63%   -650     -471   

  Fivedimes Bookmaker BetOnline  ...    Jazz Sportsinteraction Betcris   Sbr  \
0      -380      -450      -400  ...    -445              -390    -450  -400   
1      +340      +351      +330  ...    +345              +300    +351  +317   
2      -200      -225      -210  ...    -215              -230    -225  -225   
3      +185      +188      +180  ...    +180              +190    +188  +188   
4      -475      -525      -465  ...    -520              -430    -525  -500   

  Wagerweb Thegreek Bodog Matchbook Looselines Bet365  
0     -400     -420  -450      -380       -445   -400  
1     +320     +330  +325      +340       +345   +320  
2     -195     -200  -210      -204       -215   -220  
3     +170     +170  +175      +181       +180   +180  
4     -525     -450  -550      -440       -520   -525  

[5 rows x 34 columns]

In [10]:
#save_df_jan_dec = dfout

1382

In [11]:
dfout.to_csv('oct_jan_2018.csv', sep=',')

## could we do this on the same site for different sports?

## next steps: turn odds into probabilities, add "outcome"/"winner" column
##                     create columns that would go into regression to find real vs consensus coef